# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#grab city weather data from WeatherPy
csv_file = "../WeatherPy/output_data/city_weather_data.csv"
weather_df = pd.read_csv(csv_file, encoding ="utf-8")
weather_df.head(15)

#clean df and get rid of unnamed column?
del weather_df["Unnamed: 0"]
weather_df.head(15)

,City,Country,Date,Max Temp,Humidity (%),Cloudiness (%),Lat,Lng,Wind Speed
0,Half Moon Bay,US,1612725589,288.71,82,1,37.4636,-122.4286,5.66
1,Ranau,MY,1612725764,298.15,94,20,5.9538,116.6641,1.03
2,Grand Forks,US,1612725575,249.26,53,75,47.9417,-97.4337,3.09
3,Northam,GB,1612725764,275.37,69,19,51.0333,-4.2167,5.81
4,Rikitea,PF,1612725358,298.33,72,91,-23.1203,-134.9692,5.50
5,Ushuaia,AR,1612725732,290.15,45,40,-54.8000,-68.3000,4.12
6,Busselton,AU,1612725527,290.37,99,98,-33.6500,115.3333,0.89
7,Jamestown,US,1612725354,265.37,67,90,42.0970,-79.2353,6.17
8,Erciş,TR,1612725764,271.15,100,40,39.0287,43.3586,1.03
9,Bethel,US,1612725719,271.48,86,90,41.3712,-73.4140,2.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#looking back at activities 3-7 for help

#get Google Map API - we are using Places API
gmaps.configure(api_key= g_key)

#set locations using lat and lng
locations = weather_df[["Lat", "Lng"]]

#set heatmap values and convert to float type - filling NaN values
humid = weather_df["Humidity (%)"].astype(float)

#set max humidity to be used in gmaps parameter below
max_humidity =humid.max()

#plot base layer of map
fig = gmaps.figure()

#make heatmap layer on top
heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating=False, max_intensity = max_humidity, 
                                 point_radius=3)

#combine the two to work with one another
fig.add_layer(heat_layer)

#show 
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#ideal vacation weather ---note that I am using K for Temp

ideal_weather_df = weather_df.loc[(weather_df["Max Temp"] > 299 ) & 
                                   (weather_df["Max Temp"] < 303) & 
                                   (weather_df["Cloudiness (%)"] < 5 ) &
                                   (weather_df["Wind Speed"] < 5) , :]
                                    
#drop na
ideal_weather_df = ideal_weather_df.dropna(how='any')


#to see how many cities my ideal weather conditions produced
ideal_weather_df.count()
                                                

City              10
Country           10
Date              10
Max Temp          10
Humidity (%)      10
Cloudiness (%)    10
Lat               10
Lng               10
Wind Speed        10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#store hotel_df
hotel_df = ideal_weather_df

#add column named Hotel Name
hotel_df["Hotel Name"]=""

#check to make sure I did it correctly
hotel_df.head(15)

,City,Country,Date,Max Temp,Humidity (%),Cloudiness (%),Lat,Lng,Wind Speed,Hotel Name
84,Port Montt,CL,1612725545,300.15,44,0,-41.4698,-72.9447,4.12,
112,Cururupu,BR,1612725776,302.49,67,1,-1.8283,-44.8683,2.83,
181,Mattru,SL,1612725784,300.62,73,3,7.6244,-11.8332,3.58,
218,São Luiz Gonzaga,BR,1612725788,302.05,46,0,-28.4083,-54.9608,3.80,
247,Ancud,CL,1612725535,301.15,34,0,-41.8697,-73.8203,3.60,
299,Kaya,BF,1612725797,302.90,14,0,13.0917,-1.0844,4.85,
370,Carutapera,BR,1612725528,302.58,67,3,-1.1950,-46.0200,4.22,
427,Port Blair,IN,1612725807,299.06,78,0,11.6667,92.7500,3.99,
473,Santiago,CL,1612725662,300.15,44,0,-33.4569,-70.6483,3.09,
485,Kibala,TD,1612725816,301.15,20,0,9.1104,18.3463,2.64,


In [15]:
# create list for hotel name
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

#for index, row in hotel_df
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    nearby_hotel = requests.get(base_url, params=params)
    
    # convert to json
    nearby_hotel = nearby_hotel.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(nearby_hotel['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

    
    

In [16]:
hotel_name


['Presidente Suites Puerto Montt',
 'Hotel Líder',
 nan,
 'Cometa Plaza Hotel',
 'Panamericana Hotel Ancud',
 'Hôtel Kaziendé',
 'HOTEL R. SOARES',
 'Symphony Samudra, Port Blair',
 'Sheraton Santiago Hotel and Convention Center',
 'Hôtel des Chasses']

In [20]:
#test it out and see if it worked
hotel_df["Hotel Name"] = hotel_name

#drop na
hotel_df.dropna()

,City,Country,Date,Max Temp,Humidity (%),Cloudiness (%),Lat,Lng,Wind Speed,Hotel Name
84,Port Montt,CL,1612725545,300.15,44,0,-41.4698,-72.9447,4.12,Presidente Suites Puerto Montt
112,Cururupu,BR,1612725776,302.49,67,1,-1.8283,-44.8683,2.83,Hotel Líder
218,São Luiz Gonzaga,BR,1612725788,302.05,46,0,-28.4083,-54.9608,3.80,Cometa Plaza Hotel
247,Ancud,CL,1612725535,301.15,34,0,-41.8697,-73.8203,3.60,Panamericana Hotel Ancud
299,Kaya,BF,1612725797,302.90,14,0,13.0917,-1.0844,4.85,Hôtel Kaziendé
370,Carutapera,BR,1612725528,302.58,67,3,-1.1950,-46.0200,4.22,HOTEL R. SOARES
427,Port Blair,IN,1612725807,299.06,78,0,11.6667,92.7500,3.99,"Symphony Samudra, Port Blair"
473,Santiago,CL,1612725662,300.15,44,0,-33.4569,-70.6483,3.09,Sheraton Santiago Hotel and Convention Center
485,Kibala,TD,1612725816,301.15,20,0,9.1104,18.3463,2.64,Hôtel des Chasses


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map

#set variable for hotel name
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))